# Data Transformation before ML takeover

In [1]:
%load_ext autoreload
%autoreload 2

## Libraries used

The libraries we will need are all listed below. Feel free to add more libraries here.

In [11]:
import requests
import json
import gzip
import shutil
import time
import os
from io import BytesIO
import pandas as pd
import datetime as dt
import boto3
import random

## Establishing our variables

**This is a non-exhaustive list of variables that we will acquire from each file that we have cleaned (will likely be more than this):**

Features used to link games to other DBs: 6

- GameID (the ID that allowed us to catch this game in particular)
- EsportsID (the Esports Tournament ID that allowed us to catch this game in particular)
- gameVersion (PATCH INFO from the first game-info entry on our ETL)
- GameDate (Get the timestamp)
- BlueTeamID (link to Teams table)
- RedTeamID (link to Teams table)

Features recorded for each game: 65

- NbWardsPlacedBlue
- NbWardsPlacedRed
- NbWardsDestroyedBlue
- NbWardsDestroyedRed
- GoldDiffAt10Top (+ = blue, - = red)
- GoldDiffAt10Jg
- GoldDiffAt10Mid
- GoldDiffAt10AD
- GoldDiffAt10Supp
- GoldDiffEndTop
- GoldDiffEndJg
- GoldDiffEndMid
- GoldDiffEndAD
- GoldDiffEndSupp
- ExpDiffAt10Top (+ = blue, - = red)
- ExpDiffAt10Jg
- ExpDiffAt10Mid
- ExpDiffAt10AD
- ExpDiffAt10Supp
- ExpDiffEndTop
- ExpDiffEndJg
- ExpDiffEndMid
- ExpDiffEndAD
- ExpDiffEndSupp
- NbCampsStolenBlue (check for True values on IsInEnemyJungle on epic_monster_killed)
- NbCampsStolenRed (check for True values)
- NbScuttlesTakenBlue
- NbScuttlesTakenRed
- NbPlatesTakenBlue
- NbPlatesTakenRed
- OuterTopBlueTimer #We mean: blue killed the outer top at 1920.54. NANs = it didn't go down.
- OuterMidBlueTimer
- OuterBotBlueTimer
- InnerTopBlueTimer
- InnerMidBlueTimer
- InnerBotBlueTimer
- BaseTopBlueTimer #Inhib turrets
- BaseMidBlueTimer
- BaseBotBlueTimer
- NexusBlueTimer1 #Two entries on this, no distinction made by Riot
- NexusBlueTimer2
- OuterTopRedTimer #We mean: red killed the outer top at 1920.54. NANs = it didn't go down.
- OuterMidRedTimer
- OuterBotRedTimer
- InnerTopRedTimer
- InnerMidRedTimer
- InnerBotRedTimer
- BaseTopRedTimer #Inhib turrets
- BaseMidRedTimer
- BaseBotRedTimer
- NexusRedTimer1 #Two entries on this, no distinction made by Riot
- NexusRedTimer2
- NbDragonsTakenBlue
- NbDragonsTakenRed
- NbElderDragonsTakenBlue
- NbElderDragonsTakenRed
- NbBaronsTakenBlue
- NbBaronsTakenRed
- NbInhibitorsTakenBlueTop
- NbInhibitorsTakenBlueMid
- NbInhibitorsTakenBlueBot
- NbInhibitorsTakenRedTop
- NbInhibitorsTakenRedMid
- NbInhibitorsTakenRedBot
- GameLength (game_end event timer)

**From this table, we will also derive a simplified version to make things compatible with Tim's LPL data. More on that later.**

## Browsing a random file example

In [20]:
games_dir = 'games'
cleaned_files = os.listdir(games_dir)

In [21]:
len(cleaned_files)

19747

In [14]:
selected_item = random.randrange(0,len(cleaned_files)-1,1)

In [16]:
selected_filename = cleaned_files[selected_item]

In [121]:
with open(f'{games_dir}/{selected_filename}','r') as game_file:
    game_json = json.load(game_file)

In [122]:
#In this notebook, the game chosen was ESPORTSTMNT01:2138327. Random being random, your choice may vary.
game_json

[{'gameDate': '2021-07-12T16:06:28.814+00:00',
  'esportsPlatformId': 'ESPORTSTMNT01:2138327',
  'gameVersion': '11.13.382.1241'},
 {'gameTime': 0.0,
  'eventType': 'game_info',
  'blue': [{'participantID': 1,
    'summonerName': 'ZRL Adiss',
    'championName': 'Sion'},
   {'participantID': 2, 'summonerName': 'ZRL Jony', 'championName': 'LeeSin'},
   {'participantID': 3,
    'summonerName': 'ZRL YouMertBro',
    'championName': 'Sylas'},
   {'participantID': 4,
    'summonerName': 'ZRL Rago44',
    'championName': 'Kalista'},
   {'participantID': 5,
    'summonerName': 'ZRL Xintox',
    'championName': 'Nautilus'}],
  'red': [{'participantID': 6,
    'summonerName': 'TP Bako',
    'championName': 'Renekton'},
   {'participantID': 7,
    'summonerName': 'TP NIKOLEX',
    'championName': 'Volibear'},
   {'participantID': 8, 'summonerName': 'TP peppe', 'championName': 'Akali'},
   {'participantID': 9, 'summonerName': 'TP KABA', 'championName': 'Tristana'},
   {'participantID': 10,
    's

**It seems that I may have gotten carried away here, as I directly started implementing functions to extract the datapoints and store them inside a dictionary.**

My code implementation then adds the dictionary into a list, then throws them into a pandas DataFrame.
From there, a CSV is created with ";" as a separator, then filed to my teammates.

In [99]:
#This part would be worth investigating later down the line if we want to further refine the data catalogging process.
#For now, I'm leaving it here as something that could have been done.

def ward_is_legitimate(coords):
    '''A ward is legitimate if it isn't placed too deep inside the enemy base.'''
    #TO_DO: think of a good function to detect wards placed in useless locations.
    
    return True

At some point in the sections below, I ran into a problem: I noticed that not all datapoints captured the Team variable correctly. So, as I counted incidences of events happening on both sides (blue and red), I ended up making a function that added numbers to the correct team using data that I gathered.

In [116]:
def add_event_to_counter(event_counter, player_index=0, team=None, team_ids_blue=None, team_ids_red=None):
    '''We get an event and assign it to the correct team.
    The code takes under account possible lapses in our data (if we don't have a 'team'
    but have the list of participants).'''
    
    changed_counter = event_counter
    team_result = team
    
    if team_result is None:
        if player_index in team_ids_blue:
            team_result = 'blue'
        elif player_index in team_ids_red:
            team_result = 'red'

    #This is not an ELSE situation. I want to run this after we attempt to fix the issue above.
    if team_result is not None:
        changed_counter[team_result.lower()]+=1

    return (changed_counter, team_result)
    

As for this, it helped me understand the default structure of a game_state event (10mn, 15mn, and end).
After running my script the first time around, I later noticed that other bugs came into play. My tweaks are available on the .py created after working on this notebook: assemble_riot_dataset.py

Note: in the python script implementation, I also accommodated for the possibility of not having "blue" or "red" logged correctly. Due to file structure differences, I had to adapt and log team-related events with "teams" keys, when my ETL process would have preferably switched them to "blue" or "red" instead.

In [93]:
def extract_game_state_data(game_event,value=0):
    '''We process the relevant status updates and add them to dictionaries to output.
    The game event and the list of participants on both teams must be provided.
    By default, without any parameter, the function will treat this event as an endgame
    event depending on the game status. Please specify either 10 or 15 on 'value' for those
    two events.'''
    dict_output = {}
    gameOver = (game_event.get('eventType') == 'game_state_end')
    value = 'End' if gameOver else value #Just a quick adjustment here.

    player_data = [i for i in game_event.get('participants') if (0 < i.get('participantID') < 11)]
    player_data = sorted(player_data, key = lambda x: x['participantID'])

    if (value == 10 or value == 15) or gameOver:

        #Initializing variables:
        gold_top_blue = player_data[0]['totalGold']
        gold_top_red = player_data[5]['totalGold']
        gold_diff_top = gold_top_blue - gold_top_red
        
        gold_jg_blue = player_data[1]['totalGold']
        gold_jg_red = player_data[6]['totalGold']
        gold_diff_jg = gold_jg_blue - gold_jg_red
        
        gold_mid_blue = player_data[2]['totalGold']
        gold_mid_red = player_data[7]['totalGold']
        gold_diff_mid = gold_mid_blue - gold_mid_red
        
        gold_ad_blue = player_data[3]['totalGold']
        gold_ad_red = player_data[8]['totalGold']
        gold_diff_ad = gold_ad_blue - gold_ad_red
        
        gold_sup_blue = player_data[4]['totalGold']
        gold_sup_red = player_data[9]['totalGold']
        gold_diff_sup = gold_sup_blue - gold_sup_red
        
        xp_diff_top = player_data[0]['XP'] - player_data[5]['XP']
        xp_diff_jg = player_data[1]['XP'] - player_data[6]['XP']
        xp_diff_mid = player_data[2]['XP'] - player_data[7]['XP']
        xp_diff_ad = player_data[3]['XP'] - player_data[8]['XP']
        xp_diff_sup = player_data[4]['XP'] - player_data[9]['XP']
        dict_output.update({
            f'GoldTopBlue{value}': gold_top_blue,
            f'GoldJgBlue{value}': gold_jg_blue,
            f'GoldMidBlue{value}': gold_mid_blue,
            f'GoldADBlue{value}': gold_ad_blue,
            f'GoldSupBlue{value}': gold_sup_blue,
            f'GoldTopRed{value}': gold_top_red,
            f'GoldJgRed{value}': gold_jg_red,
            f'GoldMidRed{value}': gold_mid_red,
            f'GoldADRed{value}': gold_ad_red,
            f'GoldSupRed{value}': gold_sup_red,
            f'GoldDiff{value}Top': gold_diff_top,
            f'GoldDiff{value}Jg': gold_diff_jg,
            f'GoldDiff{value}Mid': gold_diff_mid,
            f'GoldDiff{value}AD': gold_diff_ad,
            f'GoldDiff{value}Bot': gold_diff_sup,
            f'XPDiff{value}Top': xp_diff_top,
            f'XPDiff{value}Jg': xp_diff_jg,
            f'XPDiff{value}Mid': xp_diff_mid,
            f'XPDiff{value}AD': xp_diff_ad,
            f'XPDiff{value}Bot': xp_diff_sup,
        })
        
        blue_kills, red_kills = game_event['blue']['championsKills'], game_event['red']['championsKills']
        blue_assists, red_assists = game_event['blue']['assists'], game_event['red']['assists']
        blue_deaths, red_deaths = game_event['blue']['deaths'], game_event['red']['deaths']
        blue_gold, red_gold = game_event['blue']['totalGold'], game_event['red']['totalGold']
        blue_tower_kills, red_tower_kills = game_event['blue']['towerKills'], game_event['red']['towerKills']
        blue_dragon_kills, red_dragon_kills = game_event['blue']['dragonKills'], game_event['red']['dragonKills']
        dict_output.update({
            f'BlueKills{value}': blue_kills,
            f'BlueAssists{value}': blue_assists,
            f'BlueDeaths{value}': blue_deaths,
            f'BlueTotalGold{value}': blue_gold,
            f'BlueDragonKills{value}': blue_dragon_kills,
            f'BlueTowerKills{value}': blue_tower_kills,
            f'RedKills{value}': red_kills,
            f'RedAssists{value}': red_assists,
            f'RedDeaths{value}': red_deaths,
            f'RedTotalGold{value}': red_gold,
            f'RedDragonKills{value}': red_dragon_kills,
            f'RedTowerKills{value}': red_tower_kills
        })
        
        if value != 10:
            blue_inhib_kills, red_inhib_kills = game_event['blue']['inhibKills'], game_event['red']['inhibKills']
            dict_output.update({
                f'BlueInhibKills{value}': blue_inhib_kills,
                f'RedInhibKills{value}': red_inhib_kills
            })
            
        if gameOver:
            blue_baron_kills, red_baron_kills = game_event['blue']['baronKills'], game_event['red']['baronKills']
            dict_output.update({
                f'BlueBaronKills{value}': blue_baron_kills,
                f'RedBaronKills{value}': red_baron_kills
            })
            
            blue_vision_score_top = player_data[0]['VISION_SCORE']
            blue_vision_score_jg = player_data[1]['VISION_SCORE']
            blue_vision_score_mid = player_data[2]['VISION_SCORE']
            blue_vision_score_ad = player_data[3]['VISION_SCORE']
            blue_vision_score_sup = player_data[4]['VISION_SCORE']
            red_vision_score_top = player_data[5]['VISION_SCORE']
            red_vision_score_jg = player_data[6]['VISION_SCORE']
            red_vision_score_mid = player_data[7]['VISION_SCORE']
            red_vision_score_ad = player_data[8]['VISION_SCORE']
            red_vision_score_sup = player_data[9]['VISION_SCORE']

            dict_output.update({
                'VisionScoreTopBlue': blue_vision_score_top,
                'VisionScoreJgBlue': blue_vision_score_jg,
                'VisionScoreMidBlue': blue_vision_score_mid,
                'VisionScoreADBlue': blue_vision_score_ad,
                'VisionScoreSupBlue': blue_vision_score_sup,
                'VisionScoreTopRed': red_vision_score_top,
                'VisionScoreJgRed': red_vision_score_jg,
                'VisionScoreMidRed': red_vision_score_mid,
                'VisionScoreADRed': red_vision_score_ad,
                'VisionScoreSupRed': red_vision_score_sup
            })

            blue_damage_dealt_top = player_data[0]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            blue_damage_dealt_jg = player_data[1]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            blue_damage_dealt_mid = player_data[2]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            blue_damage_dealt_ad = player_data[3]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            blue_damage_dealt_sup = player_data[4]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_top = player_data[5]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_jg = player_data[6]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_mid = player_data[7]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_ad = player_data[8]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_sup = player_data[9]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']

            dict_output.update({
                'DamageDealtTopBlue': blue_damage_dealt_top,
                'DamageDealtJgBlue': blue_damage_dealt_jg,
                'DamageDealtMidBlue': blue_damage_dealt_mid,
                'DamageDealtADBlue': blue_damage_dealt_ad,
                'DamageDealtSupBlue': blue_damage_dealt_sup,
                'DamageDealtTopRed': red_damage_dealt_top,
                'DamageDealtJgRed': red_damage_dealt_jg,
                'DamageDealtMidRed': red_damage_dealt_mid,
                'DamageDealtADRed': red_damage_dealt_ad,
                'DamageDealtSupRed': red_damage_dealt_sup
            })
            
            blue_damage_taken_top = player_data[0]['TOTAL_DAMAGE_TAKEN']
            blue_damage_taken_jg = player_data[1]['TOTAL_DAMAGE_TAKEN']
            blue_damage_taken_mid = player_data[2]['TOTAL_DAMAGE_TAKEN']
            blue_damage_taken_ad = player_data[3]['TOTAL_DAMAGE_TAKEN']
            blue_damage_taken_sup = player_data[4]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_top = player_data[5]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_jg = player_data[6]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_mid = player_data[7]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_ad = player_data[8]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_sup = player_data[9]['TOTAL_DAMAGE_TAKEN']

            dict_output.update({
                'DamageTakenTopBlue': blue_damage_taken_top,
                'DamageTakenJgBlue': blue_damage_taken_jg,
                'DamageTakenMidBlue': blue_damage_taken_mid,
                'DamageTakenADBlue': blue_damage_taken_ad,
                'DamageTakenSupBlue': blue_damage_taken_sup,
                'DamageTakenTopRed': red_damage_taken_top,
                'DamageTakenJgRed': red_damage_taken_jg,
                'DamageTakenMidRed': red_damage_taken_mid,
                'DamageTakenADRed': red_damage_taken_ad,
                'DamageTakenSupRed': red_damage_taken_sup
            })

            blue_cc_score_top = player_data[0]['TIME_CCING_OTHERS']
            blue_cc_score_jg = player_data[1]['TIME_CCING_OTHERS']
            blue_cc_score_mid = player_data[2]['TIME_CCING_OTHERS']
            blue_cc_score_ad = player_data[3]['TIME_CCING_OTHERS']
            blue_cc_score_sup = player_data[4]['TIME_CCING_OTHERS']
            red_cc_score_top = player_data[5]['TIME_CCING_OTHERS']
            red_cc_score_jg = player_data[6]['TIME_CCING_OTHERS']
            red_cc_score_mid = player_data[7]['TIME_CCING_OTHERS']
            red_cc_score_ad = player_data[8]['TIME_CCING_OTHERS']
            red_cc_score_sup = player_data[9]['TIME_CCING_OTHERS']

            dict_output.update({
                'TotalCCDurationTopBlue': blue_cc_score_top,
                'TotalCCDurationJgBlue': blue_cc_score_jg,
                'TotalCCDurationMidBlue': blue_cc_score_mid,
                'TotalCCDurationADBlue': blue_cc_score_ad,
                'TotalCCDurationSupBlue': blue_cc_score_sup,
                'TotalCCDurationTopRed': red_cc_score_top,
                'TotalCCDurationJgRed': red_cc_score_jg,
                'TotalCCDurationMidRed': red_cc_score_mid,
                'TotalCCDurationADRed': red_cc_score_ad,
                'TotalCCDurationSupRed': red_cc_score_sup
            })            

    return dict_output
        
        

Below is the notebook script that I made to extract all events from a random game of a random choosing. Perhaps I should have inspected other games, but this exploration came in handy in the live implementation anyway as only 101 files out of 19k+ failed to oblige.

In [119]:
dict_stats = {}
game_info_dict = {}
game_state_10 = {}
game_state_15 = {}
game_state_end = {}
header_stats = None
wards_placed = [0 for i in range(0,10)]
control_wards_placed = [0 for i in range(0,10)]
wards_killed = [0 for i in range(0,10)]
control_wards_killed = [0 for i in range(0,10)]
current_event = None
previous_event = None
game_duration = 7200 #Put it at two hours for funsies.
game_winner = None
dragon_type_queued = [None, None]
dragon_index = 0
dragon_soul_taken = False
dragon_soul = {
    'DragonSoulTimer':None,
    'DragonSoulType':None,
    'DragonSoulTaker':None
}
default_dict = {'blue':0,'red':0}
elder_dragons_taken = {'blue':0,'red':0}
own_camps_taken = {'blue':0,'red':0}
enemy_camps_taken = {'blue':0,'red':0}
rift_herald_count = {'blue':0,'red':0}
scuttle_count = {'blue':0,'red':0}
dragon_count = {'blue':0,'red':0}
dragon_type_count = {'blue':{}, 'red':{}}
baron_count = {'blue':0,'red':0}
tower_count = {'blue':0,'red':0}
tower_log = {
    'OuterTopBlueTimer': None,
    'OuterMidBlueTimer': None,
    'OuterBotBlueTimer': None,
    'InnerTopBlueTimer': None,
    'InnerMidBlueTimer': None,
    'InnerBotBlueTimer': None,
    'BaseTopBlueTimer': None,
    'BaseMidBlueTimer': None,
    'BaseBotBlueTimer': None,
    'Nexus1MidBlueTimer': None,
    'Nexus2MidBlueTimer': None,
    'OuterTopRedTimer': None,
    'OuterMidRedTimer': None,
    'OuterBotRedTimer': None,
    'InnerTopRedTimer': None,
    'InnerMidRedTimer': None,
    'InnerBotRedTimer': None,
    'BaseTopRedTimer': None,
    'BaseMidRedTimer': None,
    'BaseBotRedTimer': None,
    'Nexus1MidRedTimer': None,
    'Nexus2MidRedTimer': None
} #Time values for each event
team_side = None

smaller_camps = ['blueCamp','redCamp','gromp','wolf','krug','raptor']

#Data integrity check, but also getting relevant data.
if game_json[-1].get('eventType',None) != 'game_end':
    print("The game's data does not contain the entirety of the game.")
    #TO_DO: plug Tim's game_end timestamp (game duration)
else:
    game_duration = game_json[-1].get('gameTime')
    game_winner = game_json[-1].get('winningTeam')
    
for game_event in game_json:
    
    # In case we have our intro header somewhere in there:
    if game_event.get('eventType',None) is None:
        
        try:
            game_date = dt.datetime.fromisoformat(game_event.get('gameDate',None))
            esports_platform = game_event.get('esportsPlatformId',None)
            game_patch = game_event.get('gameVersion',None)
            header_stats = {
                'gameDate': game_date,
                'esportsPlatformId': esports_platform,
                'gameVersion': game_patch
            }
        except:
            print(f'This game has an event without a label other than the game descriptors. Please inspect {selected_filename} later.')
    else:
        current_event = game_event.get('eventType',None)

    # Important bit: we get our game info.
    if game_event.get('eventType') == 'game_info':
        blue_participant_id = []
        red_participant_id = []
        blue_summoner_name = []
        red_summoner_name = []
        blue_champion = []
        red_champion = []
        
        blue_data = game_event.get('blue')
        for player in blue_data:
            blue_participant_id.append(player.get('participantID'))
            blue_summoner_name.append(player.get('summonerName'))
            blue_champion.append(player.get('championName'))

        red_data = game_event.get('red')
        for player in red_data:
            red_participant_id.append(player.get('participantID'))
            red_summoner_name.append(player.get('summonerName'))
            red_champion.append(player.get('championName'))

    if game_event.get('eventType') == 'ward_placed':
        placer = game_event.get('placer')
        #location = game_event.get('position') #Unused for now
        ward_type = game_event.get('wardType')

        #We verify that the ward isn't placed when the game is nearly done
        #Alternatively, we could check that the ward isn't placed within a
        #set radius of the base or nexus. Said radius must reflect that the ward
        #is useless. That said, we won't implement that for now, for lack
        #of knowledge.
        if game_event.get('gameTime') < game_duration-20:
            #Is our ward a control ward?
            #Keep in mind: indexes start from 0, the -1 adapts it all.
            if ward_type == 'control':
                control_wards_placed[placer-1] = control_wards_placed[placer-1] + 1
            else:
                wards_placed[placer-1] = wards_placed[placer-1] + 1                

    if game_event.get('eventType') == 'ward_killed':
        killer = game_event.get('killer')
        #location = game_event.get('position') #Unused for now
        ward_type = game_event.get('wardType')

        #Wards killed
        if game_event.get('gameTime') < game_duration-20:
            #Is our ward a control ward?
            #Keep in mind: indexes start from 0, the -1 adapts it all.
            if ward_type == 'control':
                control_wards_killed[killer-1] = control_wards_killed[killer-1] + 1
            else:
                wards_killed[killer-1] = wards_killed[killer-1] + 1   
    
    if game_event.get('eventType') == 'queued_dragon_info':
        dragon_index = dragon_index + 1
        if dragon_type_queued[0] is None:
            dragon_type_queued[0] = game_event.get('nextDragonName')
        else:
            dragon_type_queued[1] = game_event.get('nextDragonName')

    if game_event.get('eventType') == 'epic_monster_kill':
        #So, there are A LOT of epic monsters killed.
        #The events have the "killer" in common.
        killer = game_event.get('killer')
        killer_team = game_event.get('team')
        
        #Step 1: counterjungle tally
        if game_event.get('inEnemyJungle'):
            #Temporary workaround given data mistakes were made
            enemy_camps_taken = add_event_to_counter(enemy_camps_taken, killer, killer_team,
                                                     blue_participant_id, red_participant_id)[0]
        #If no counterjungling is afoot, take this number instead:
        elif game_event.get('monsterType') in smaller_camps:
            own_camps_taken = add_event_to_counter(own_camps_taken, killer, killer_team,
                                                     blue_participant_id, red_participant_id)[0]

        #Step 2: Scuttles, Heralds, Barons, and Dragons
        if game_event.get('monsterType') == 'scuttleCrab':
            scuttle_count = add_event_to_counter(scuttle_count, killer, killer_team,
                                                     blue_participant_id, red_participant_id)[0]

        if game_event.get('monsterType') == 'riftHerald':
            rift_herald_count = add_event_to_counter(rift_herald_count, killer, killer_team,
                                                     blue_participant_id, red_participant_id)[0]

        if game_event.get('monsterType') == 'baron':
            baron_count = add_event_to_counter(baron_count, killer, killer_team,
                                                     blue_participant_id, red_participant_id)[0]

        if game_event.get('monsterType') == 'dragon':
            dragon_count, team_side = add_event_to_counter(dragon_count, killer, killer_team,
                                                     blue_participant_id, red_participant_id)
            if team_side is not None:
                side_select = {'blue': 0, 'red':1}
                if not dragon_soul_taken:
                    i = side_select.get(team_side)
                    nb_dragons = dragon_count[i]
                    if nb_dragons == 4:
                        dragon_soul_taken = True
                        dragon_soul = {
                            'DragonSoulTimer':game_event.get('gameTime'),
                            'DragonSoulType':dragon_type_queued[0],
                            'DragonSoulTeam':team_side
                        }
                else:
                    elder_dragons_taken = add_event_to_counter(elder_dragons_taken, killer, killer_team,
                                                     blue_participant_id, red_participant_id)[0]
                        
                
                dragon_type_count[team_side].update({dragon_number+1 : [game_event.get('gameTime'), dragon_type_queued[0]]})
                
            if dragon_type_queued[1] is not None:
                dragon_type_queued[0] = dragon_type_queued[1]
                dragon_type_queued[1] = None

    if game_event.get('eventType') == 'building_destroyed':
        try:
            team_side = game_event.get('team',None)
            building_type = game_event.get('buildingType')
            
            if building_type == 'turret' and team_side is not None:
                building_lane = game_event.get('lane').title()
                building_tier = game_event.get('turretTier').title()
                
                if building_tier == 'Nexus':
                    if tower_log.get(f'{building_tier}1{building_lane}{team_side.title()}Timer',None) is not None:
                        building_tier = 'Nexus2'
                    else:
                        building_tier = 'Nexus1'
                        
                tower_log.update({f'{building_tier}{building_lane}{team_side.title()}Timer':
                                  game_event.get('gameTime')})
                tower_count = add_event_to_counter(tower_count, None, team_side,
                                                     None, None)[0]
        except:
            print('Did a building self-destruct by any chance? Something has gone awry here.')

    if game_event.get('eventType') == 'game_state_10mn':
        game_state_10 = extract_game_state_data(game_event,10)

    if game_event.get('eventType') == 'game_state_15mn':
        game_state_15 = extract_game_state_data(game_event,15)
    
    if game_event.get('eventType') == 'game_state_end':
        game_state_end = extract_game_state_data(game_event)
        
    previous_event = current_event
    
        
#This will be our first entry in the dictionaries.
if header_stats is not None:
    dict_stats.update(header_stats)

dict_stats.update({
    'NbWardsPlacedBlue': sum(wards_placed[i-1] for i in blue_participant_id),
    'NbWardsPlacedRed': sum(wards_placed[i-1] for i in red_participant_id),
    'NbControlWardsPlacedBlue': sum(control_wards_placed[i-1] for i in blue_participant_id),
    'NbControlWardsPlacedRed': sum(control_wards_placed[i-1] for i in red_participant_id),
    'NbWardsKilledBlue': sum(wards_killed[i-1] for i in blue_participant_id),
    'NbWardsKilledRed': sum(wards_killed[i-1] for i in red_participant_id),
    'NbControlWardsKilledBlue': sum(control_wards_killed[i-1] for i in blue_participant_id),
    'NbControlWardsKilledRed':sum(control_wards_killed[i-1] for i in red_participant_id)
})

dict_stats.update({
    'NbCampsSecuredBlue': own_camps_taken['blue'],
    'NbCampsSecuredRed': own_camps_taken['red'],
    'NbCampsStolenBlue': enemy_camps_taken['blue'],
    'NbCampsStolenRed': enemy_camps_taken['red'],
    'NbScuttlesBlue': scuttle_count['blue'],
    'NbScuttlesRed': scuttle_count['red'],
    'NbRiftHeraldsBlue': rift_herald_count['blue'],
    'NbRiftHeraldsRed': rift_herald_count['red'],
    'NbDragonsBlue': dragon_count['blue']-elder_dragons_taken['blue'], #Denotes only regular dragons
    'NbDragonsRed': dragon_count['red']-elder_dragons_taken['red'],
    'NbBaronsBlue': baron_count['blue'],
    'NbBaronsRed': baron_count['red'],
    'NbEldersBlue': elder_dragons_taken['blue'],
    'NbEldersRed': elder_dragons_taken['red'],
    'NbTowersBlue': tower_count['red'],
    'NbTowersRed': tower_count['blue']
})

dict_stats.update(tower_log)
dict_stats.update(dragon_soul)
dict_stats.update(game_state_10)
dict_stats.update(game_state_15)
dict_stats.update(game_state_end)
dict_stats.update({'winner': game_winner})

#dict_stats.update({'MoreDragonInfo':dragon_type_count})

In [120]:
#A sample output for our routine, to be fed to a DataFrame.
dict_stats

{'gameDate': datetime.datetime(2021, 7, 12, 16, 6, 28, 814000, tzinfo=datetime.timezone.utc),
 'esportsPlatformId': 'ESPORTSTMNT01:2138327',
 'gameVersion': '11.13.382.1241',
 'NbWardsPlacedBlue': 61,
 'NbWardsPlacedRed': 44,
 'NbControlWardsPlacedBlue': 28,
 'NbControlWardsPlacedRed': 26,
 'NbWardsKilledBlue': 8,
 'NbWardsKilledRed': 12,
 'NbControlWardsKilledBlue': 15,
 'NbControlWardsKilledRed': 19,
 'NbCampsSecuredBlue': 0,
 'NbCampsSecuredRed': 0,
 'NbCampsStolenBlue': 2,
 'NbCampsStolenRed': 0,
 'NbScuttlesBlue': 0,
 'NbScuttlesRed': 0,
 'NbRiftHeraldsBlue': 0,
 'NbRiftHeraldsRed': 0,
 'NbDragonsBlue': 0,
 'NbDragonsRed': 0,
 'NbBaronsBlue': 0,
 'NbBaronsRed': 0,
 'NbEldersBlue': 0,
 'NbEldersRed': 0,
 'NbTowersBlue': 3,
 'NbTowersRed': 11,
 'OuterTopBlueTimer': 1112.701,
 'OuterMidBlueTimer': 1077.287,
 'OuterBotBlueTimer': 726.987,
 'InnerTopBlueTimer': 1134.966,
 'InnerMidBlueTimer': 1490.756,
 'InnerBotBlueTimer': 1604.78,
 'BaseTopBlueTimer': 1509.052,
 'BaseMidBlueTimer': 1